# Load Head with id2label

In [ ]:
!pip install -Uq adapters

First, we create the model and load the trained adapter and head. With the head, the labels, and the i2label mapping is automatically loaded. With `model.get_labels()` and `model.get_labels_dict()` you can access them.

In [11]:
from adapters import AutoAdapterModel, AdapterConfigBase
from transformers import AutoTokenizer
import numpy as np

model_name = "bert-base-uncased"
model = AutoAdapterModel.from_pretrained(model_name)
adapter_name = model.load_adapter("ner/conll2003@ukp")

model.active_adapters = adapter_name
model.active_head = "ner_head"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# How you can acces the labels and the mapping for a pretrained head
print(model.get_labels())
print(model.get_labels_dict())

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertAdapterModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O']
{1: 'B-LOC', 7: 'B-MISC', 5: 'B-ORG', 3: 'B-PER', 2: 'I-LOC', 8: 'I-MISC', 6: 'I-ORG', 4: 'I-PER', 0: 'O'}


This helper function allows us to get the sequence of ids of the predicted output for a specific input sentence.

In [12]:
def predict(sentence):
  tokens = tokenizer.encode(
        sentence,
        return_tensors="pt",
    )
  model.eval()
  preds = model(tokens, adapter_names=['ner'])[0]
  preds = preds.detach().numpy()
  preds = np.argmax(preds, axis=2)
  return tokenizer.tokenize(sentence), preds.squeeze()[1:-1] 

If we want to use the model to predict the labels of a sentence we can use the `model.get_labels_dict()` function to map the predicted label ids to the corresponding label, as for the example text.

In [13]:
example_text="Germany's representative to the European Union\'s veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer."
# get the mapping of ids to labels
label_map = model.get_labels_dict()
tokens, preds = predict(example_text)
for token, pred in zip(tokens, preds):
  print(f"{token}({label_map[pred]}) ", end="")

germany(B-LOC) '(O) s(O) representative(O) to(O) the(O) european(B-ORG) union(I-ORG) '(O) s(O) veterinary(B-ORG) committee(I-ORG) werner(B-PER) z(I-PER) ##wing(I-PER) ##mann(I-PER) said(O) on(O) wednesday(O) consumers(O) should(O) buy(O) sheep(O) ##me(O) ##at(O) from(O) countries(O) other(O) than(O) britain(B-LOC) until(O) the(O) scientific(O) advice(O) was(O) clearer(O) .(O) 